# CH04 The SOLID Principles

s: 단일 책임 원칙  
o: 개방/폐쇄의 원칙  
L: 리스코프(Liskov) 치환 원칙  
I: 인터페이스 분리 원칙  
D: 의존성 역전 원칙  

##### 목표
- 소프트우에어 디자인에서의 SOLID 원칙을 익힌다
- 단일 책임 원칙을 따르는 컴포넌트를 디자인한다.
- 개방/패쇠의 ㅜ언칙을 통해 보다 유지보수성을 뛰어나게 한다.
- 리스코프 치환원칙을 준수하여 객체지향 디자인에서 적절한 클래스 계층을 설계한다.
- 인터페이스 분리와 의존성 역전을 활용해 설계하기

#### 1. 단일 책임 원칙(Single Responsiblity Principl - SRP)
: 소프트웨어 컴포넌트(일반적으로 클래스)가 단 하나의 책임을 저야한다는 원칙


1-1.너무 많은 책임을 가진 클래스

로그 파일이나 데이터베이스와 같은 소스에서 이벤트의 정보를 읽어서 로그별로 필요한 액션을 분류하는 애플리케이션을 예시 코드

In [2]:
# srp_1.py
class SystemMonitor:
    def load_activity(self):
        """소스에서 처리할 이벤트를 가져오기"""

    def identify_events(self):
        """가져온 데이터를 파싱하여 도메인 객체 이벤트로 변환"""

    def stream_events(self):
        """파싱한 이벤트를 외부 에이전트로 전송"""

1-2. 책임 분산

솔루션을 보다 쉽게 관리하기 위해 모든 메서드를 다른 클래스로 분리하여 각 클래스마다 단일 책임을 갖게 하자.

In [ ]:
class ActivityReader:
    def load(self):
        pass
class SystemMonitor:
    def identify_events(self):
        pass
class Ouput:
    def stream(self):
        pass
    
class AlertSystem:
    def run(self):
        ActivityReader().load()
        SystemMonitor().identify_events()
        Output().stream()

#### 2. 개방/ 폐쇄 원칙
:개방/폐쇄 원칙(Open/Close Principle)은 모듈이 개방되어 있으면서도 폐쇄되어야 한다는 원칙.

클래스를 디자인할 때는 유지보수가 쉽도록 로직을 캡슐화하여 확장에는 개방되고 수정에는 폐쇄되도록 해야 한다.

간단히 말해서 확장 가능하고, 새로운 요구사항이나 도메인 변화에 잘 적응하는 코드를 작성해야 한다는 뜻. 새로운 문제가 발생할 경우 추가만 할 뿐 기존 코드는 그대로 유지해야 한다.

새로운 기능을 추가하다가 기존 코드를 수정했다면 기존 로직이 잘못 디자인되었다는 것을 뜻.

#### 2-1. 개방/패쇄 원칙을 따르지 않을 경우 유지보수의 어려움  

다른 시스템에서 발생하는 이벤트를 분류하는 기능을 가진 클래스가 있다.  
각 컴포넌트는 수집한 데이터를 기반으로 어떤 타입의 이벤트인지 정확히 분류해야 한다.

In [4]:
# openclosed_1.py
class Event:
    def __init__(self, raw_data):
        self.raw_data = raw_data


class UnknownEvent(Event):
    """데이터만으로 식별할 수 없는 이벤트"""


class LoginEvent(Event):
    """로그인 사용자에 의한 이벤트"""


class LogoutEvent(Event):
    """로그아웃 사용자에 대한 이벤트"""


class SystemMonitor:
    """시스템에서 발생한 이벤트 분류"""

    def __init__(self, event_data):
        self.event_data = event_data

    def identify_event(self):
        if (
            self.event_data["before"]["session"] == 0
            and self.event_data["after"]["session"] == 1
        ):
            return LoginEvent(self.event_data)
        elif (
            self.event_data["before"]["session"] == 1
            and self.event_data["after"]["session"] == 0
        ):
            return LogoutEvent(self.event_data)

        return UnknownEvent(self.event_data)

In [5]:
l1 = SystemMonitor({"before": {"session": 0}, "after": {"session": 1}})
l1.identify_event().__class__.__name__

'LoginEvent'

In [6]:
l2 = SystemMonitor({"before": {"session": 1}, "after": {"session": 0}})
l2.identify_event().__class__.__name__

'LogoutEvent'

In [7]:
l3 = SystemMonitor({"before": {"session": 1}, "after": {"session": 1}})
l3.identify_event().__class__.__name__

'UnknownEvent'

#### 2-2 확장성을 가진 이벤트 시스템으로 리팩토링

:이전 예제의 문제점은 SystemMonitor 클래스가 분류하려는 구체 클래스와 직접 상호 작용한다는 점이다. 개방/폐쇄 원칙을 따르는 디자인을 달성하려면 추상화를 해야 한다.

SystemMonitor 클래스를 추상적인 이벤트와 협력하도록 변경하고, 이벤트에 대응하는 개별 로직은 각 이벤트 클래스에 위임하는 것이다.  

그런 다음 각각의 이벤트에 다형성을 가진 새로운 메서드를 추가해야 한다. 이 메서드는 전달되는 데이터가 해당 클래스의 타입과 일치하는지 판단하는 역할을 한다. 또한 기존 분류 로직을 수정하여 이 메서드를 사용해 전체 이벤트를 돌면서 검사해야 한다.


In [8]:
class Event:
    def __init__(self, raw_data):
        self.raw_data = raw_data

    @staticmethod
    def meets_condition(event_data: dict):
        return False


class UnknownEvent(Event):
    """데이터만으로 식별할 수 없는 이벤트"""


class LoginEvent(Event):
    @staticmethod
    def meets_condition(event_data: dict):
        return (
            event_data["before"]["session"] == 0
            and event_data["after"]["session"] == 1
        )


class LogoutEvent(Event):
    @staticmethod
    def meets_condition(event_data: dict):
        return (
            event_data["before"]["session"] == 1
            and event_data["after"]["session"] == 0
        )


class SystemMonitor:
    """시스템에서 발생한 이벤트 분류"""

    def __init__(self, event_data):
        self.event_data = event_data

    def identify_event(self):
        for event_cls in Event.__subclasses__():
            try:
                if event_cls.meets_condition(self.event_data):
                    return event_cls(self.event_data)
            except KeyError:
                continue
        return UnknownEvent(self.event_data)

이제 상호 작용이 추상화를 통해 이뤄지고 있음에 주목하자. 분류 메서드는 이제 특정 이벤트 타입 대신에 일반적인 인터페이스를 따르는 제네릭 이벤트와 동작한다. 이 인터페이스를 따르는 제네릭들은 모두 meets_condition 메서드를 구현하여 다형성을 보장한다.

 

__subclasses__() 메서드를 사용해 이벤트 유형을 찾는 것에 주목하자. 이제 새로운 유형의 이벤트를 지원하려면 단지 Event 클래스를 상속받아 비즈니스 로직에 따라 meets_condition() 메서드를 구현하기만 하면 된다.

#### 2-3 이벤트 시스템 확장

위의 개선된 디자인이 실제로 원하는 대로 확장 가능한지 증명해본다. 새로운 요구사항이 생겨서 모니터링 중인 시스템의 사용자 트랜잭션에 대응하는 이벤트를 지원해야 한다고 가정한다.

TransactionEvent 라는 새로운 클래스를 추가하는 것만으로 기존 코드가 예상한 대로 잘 작동한다.

In [9]:
class Event:
    def __init__(self, raw_data):
        self.raw_data = raw_data

    @staticmethod
    def meets_condition(event_data: dict):
        return False


class UnknownEvent(Event):
    """데이터만으로 식별할 수 없는 이벤트"""


class LoginEvent(Event):
    @staticmethod
    def meets_condition(event_data: dict):
        return (
            event_data["before"]["session"] == 0
            and event_data["after"]["session"] == 1
        )


class LogoutEvent(Event):
    @staticmethod
    def meets_condition(event_data: dict):
        return (
            event_data["before"]["session"] == 1
            and event_data["after"]["session"] == 0
        )


class TransactionEvent(Event):
    """시스템에서 발생한 트랜잭션 이벤트"""

    @staticmethod
    def meets_condition(event_data: dict):
        return event_data["after"].get("transaction") is not None


class SystemMonitor:
    """시스템에서 발생한 이벤트 분류"""

    def __init__(self, event_data):
        self.event_data = event_data

    def identify_event(self):
        for event_cls in Event.__subclasses__():
            try:
                if event_cls.meets_condition(self.event_data):
                    return event_cls(self.event_data)
            except KeyError:
                continue
        return UnknownEvent(self.event_data)

In [11]:
l1 = SystemMonitor({"before": {"session": 0}, "after": {"session": 1}})
l1.identify_event().__class__.__name__

'LoginEvent'

In [12]:
l2 = SystemMonitor({"before": {"session": 1}, "after": {"session": 0}})
l2.identify_event().__class__.__name__

'LogoutEvent'

In [13]:
l3 = SystemMonitor({"before": {"session": 1}, "after": {"session": 1}})
l3.identify_event().__class__.__name__

'UnknownEvent'

In [14]:
l4 = SystemMonitor({"after": {"transaction": "Tx001"}})
l4.identify_event().__class__.__name__

'TransactionEvent'

#### OCP 최종 정리
OCP 는 다형성의 효과적인 사용과 밀접하게 관련되어 있다. 다형성을 따르는 형태의 계약을 만들고 모델을 쉽게 확장할 수 있는 일반적인 구조로 디자인하는 것이다.

 

이 원칙은 유지보수성에 대한 문제를 해결한다. OCP 를 따르지 않으면 파급 효과가 생기거나 작은 변경이 코드 전체에 영향을 미치거나 다른 부분을 손상시키게 된다.

 

마지막으로 코드를 변경하지 않고 기능을 확장하기 위해서는 보호하려는 추상화(예제에서는 새로운 이벤트 유형)에 대해서 적절하게 폐쇄를 해야 한다는 것이다. 그러나 항상 적용 가능한 것은 아니다. 특정 요구 사항에 대해 적절한 추상화가 다른 유형의 요구사항에 대해서는 그렇지 않을 수 있다. 이러한 경우는 가장 확장 가능한 요구사항에 적합한 폐쇄를 선택해야 한다.


#### 3. 리스코프 치환 원칙(LSP)
: 리스코프 치환 원칙(Liskov Subsititution Principle, LSP) 은 설계 시 안정성을 유지하기 위해 객체 타입이 유지해야 하는 일련의 특성을 말한다.

 

LSP 는 어떤 클래스에서든 클라이언트는 특별한 주의를 기울이지 않고도 하위 타입을 사용할 수 있어야 함을 뜻한다. 즉, 클라이언트는 완전히 분리되어 있으며 클래스 변경 사항과 독립되어야 한다.

 

공식적인 정의는 다음과 같다.


- 만약 S가 T의 하위 타입이라면 프로그램을 변경하지 않고 T 타입의 객체를 S 타입의 객체로 치환 가능해야 한다.

#### 도구를 사용해 LSP 문제 검사하기

 코드포매팅과도구"에서 소개한My py나Pglint 같은도구 를사용해쉽게검출할수있다.

- ##### 메서드 3서명의 잘못된 데이터타입 검사

드전체에타입어노테이션을사용했고 Mypy 를설정했다면초기에기본오류여부와LSP준수여부를빠르게확인할수있다.  

Event 클래스의하위클래스중하나가호환되지않는방식으로메서드를재정의하면Mypy는 어노테이션을검사하여이를확인한다.


In [16]:
class Event:
    ...

    def meets_condition(self, event_data: dict) -> bool:
        return False


class LoginEvent(Event):
    def meets_condition(self, event_data: list) -> bool:
        return bool(event_data)


class LogoutEvent(Event):
    def meets_condition(self, event_data: dict, override: bool) -> bool:
        if override:
            return True
        ...
        
        


error: Argument 1 of "meets_condition" incompatible with supertype "Event"

type : ignore 같은 주석을 통해 에러를 무시하면 안 된다. 포매팅 도구를 통해.  
 
 보고된 내용은 모두 수정이 필요한 타당한 이유를 갖고 있는 내용이기 때문이다.

#####  Pylint로호환되지않는서명검사
:LSP 위반사례는계층의파라미터타입이다른것이아니라메서드의 서명자체가완전히다른경우이다. 이것은아주큰실수처럼보이지만탐지하는것은쉽지않 다. 파이썬은인터프리터언어이므로초기에컴파일러를사용해이러한유형의오류를감지하 지못했다면런타임까지발견되지않는다. 다행히도Mypy나Pylint와같은정적코드분석기 를사용해초기에이러한오류를잡을수있다.

In [ ]:
class LogoutEvent(Event):
    def meets_condition(self, event_data: dict, override: bool) -> bool:
        if override:
            return True
        ...
##Parameters differ from overridden 'meets_condition' method (arguments-differ)

#### 3-2.  애매한 LSP 위반 사례
: 어떤 경우에는 LSP 를 위반한 것이 명확하지 않아서 자동화된 도구로 검사하기 애매할 수 있다.

DbC 에서 계약이 수정되는 경우는 특히 자동으로 감지하기가 더 어렵다.  
부모 클래스는 클라이언트와의 계약을 정의한다.  
하위 클래스는 그 계약을 따라야 한다.  
예를 들면 아래와 같은 조건을 따라야 한다고 해보자.

- 하위 클래스는 부모 클래스에 정의된 것보다 사전조건을 엄격하게 만들면 안된다.
- 하위 클래스는 부모 클래스에 정의된 것보다 약한 사후조건을 만들면 안된다.

In [19]:
#lsp_2.py           BAD CASE
class Event:
    def __init__(self, raw_data):
        self.raw_data = raw_data

    @staticmethod
    def meets_condition(event_data: dict):
        return False

    @staticmethod
    def meets_condition_pre(event_data: dict):
        """인터페이스 계약의 사전조건
        ``event_data`` 파라미터가 적절한 형태인지 유효성 검사
        """
        assert isinstance(event_data, dict), f"{event_data!r} is not a dict"
        for moment in ("before", "after"):
            assert moment in event_data, f"{moment} not in {event_data}"
            assert isinstance(event_data[moment], dict)

예외는 캡슐화를 약화시킨다. 예외 처리를 하려면 호출하는 객체에 대한 부가적인 정보가 필요하기 때문이다.

이제 올바른 이벤트 유형을 확인하기 전에 사전조건을 먼저 검사

In [20]:
class SystemMonitor:
    """시스템에서 발생한 이벤트 분류"""

    def __init__(self, event_data):
        self.event_data = event_data

    def identify_event(self):
        Event.meets_condition_pre(self.event_data)
        event_cls = next(
            (
                event_cls
                for event_cls in Event.__subclasses__()
                if event_cls.meets_condition(self.event_data)
            ),
            UnknownEvent,
        )
        return event_cls(self.event_data)

계약은 오직 최상위 레벨의 키 "before", "after" 가 필수이고, 그 값 또한 사전 타입이어야 한다고만 명시되어 있다.  
하위 클래스에서 보다 제한적인 파라미터를 요구하는 경우 검사에 통과하지 못한다.

앞서 만든 트랜잭션 이벤트 클래스는 올바르게 설계되었다. "transaction" 이라는 키에 제한을 두지 않고 사용하고 있다.  
그 값이 있을 경우에만 사용하고 필수로 꼭 필요한 것은 아니다.

In [21]:
class TransactionEvent(Event):
    """시스템에서 발생한 트랜잭션 이벤트"""

    @staticmethod
    def meets_condition(event_data: dict):
        return event_data["after"].get("transaction") is not None

그러나 LoginEvent 와 LogoutEvent 클래스는 before, after 의 "session" 이라는 키를 사용하기 때문에 그대로 사용할 수 없다. 이렇게 되면 KeyError 가 발생하기 때문에 나머지 클래스를 사용하는 것과 같은 방식으로 클래스를 사용할 수 없다.

 

이 문제는 TransactionEvent 와 마찬가지로 대괄호 대신 .get() 메서드를 사용하여 해결할 수 있다.

In [22]:
class LoginEvent(Event):
    @staticmethod
    def meets_condition(event_data: dict):
        return (
            event_data["before"].get("session") == 0
            and event_data["after"].get("session") == 1
        )


class LogoutEvent(Event):
    @staticmethod
    def meets_condition(event_data: dict):
        return (
            event_data["before"].get("session") == 1
            and event_data["after"].get("session") == 0
        )

In [23]:
l1 = SystemMonitor({"before": {"session": 0}, "after": {"session": 1}})
l1.identify_event().__class__.__name__

'LoginEvent'

In [24]:
l2 = SystemMonitor({"before": {"session": 1}, "after": {"session": 0}})
l2.identify_event().__class__.__name__

'LogoutEvent'

In [25]:
l3 = SystemMonitor({"before": {"session": 1}, "after": {"session": 1}})
l3.identify_event().__class__.__name__

'UnknownEvent'

In [26]:
l4 = SystemMonitor({"before": {}, "after": {"transaction": "Tx001"}})
l4.identify_event().__class__.__name__

'TransactionEvent'

자동화된 도구가 이런 부분까지 검출해주기를 기대하는 것은 다시 무리가 있다.  
 클래스 디자인을 할 때는 실수로 메서드의 입력과 출력을 변경해서 원래 기대한 것과 달라지지 않도록 주의해야 한다.

#### 3-3. LSP 최종 정리
LSP 는 객체지향 소프트웨어 설계의 핵심이 되는 다형성을 강조하기 때문에 좋은 디자인의 기초가 된다. 인터페이스의 메서드가 올바른 계층구조를 갖도록 하여 상속된 클래스가 부모 클래스와 다형성을 유지하도록 하는 것이다.

 

이는 앞선 OCP 와도 관련되어 있다. 새로운 클래스가 원래의 계약과 호환되지 않는 확장을 하려고 하면 클라이언트와의 계약이 깨져서 결과적으로 그러한 확장이 불가능하다. 또는 확장을 가능하게 하려면 수정에 대해 폐쇄되어야 한다는 원칙을 깨야 한다.

 

LSP 에서 제안하는 방식으로 신중하게 클래스를 디자인하면 계층을 올바르게 확장하는데 도움이 된다. 즉 LSP 가 OCP에 기여한다고 볼 수 있다.

#### 4. 인터페이스 분리 원칙
:인터페이스 분리 원칙(Interface Segregation Principle, ISP) 은 작은 인터페이스" 에 대한 가이드라인을 제공한다.

객체 지향적인 용어로 인터페이스는 객체가 노출하는 메서드의 집합이다. 이는 다른 클라이언트에서 호출할 수 있느 요청들이다. 인터페이스는 클래스에 노출된 동작의 정의와 구현을 분리한다.

 

파이썬에서 인터페이스는 클래스 메서드의 형태를 보고 암시적으로 정의된다. 파이썬이 덕 타이핑(duck typing)을 따르기 때문이다.

 

덕 타이핑(duck typing) 은 모든 객체가 자신이 가지고 있는 메서드와 자신이 할 수 있는 일에 의해서 표현된다는 점에서 출발한다. 즉 클래스의 유형, 이름, docstring, 클래스 속성 또는 인스턴스 속성에 관계없이 객체의 본질을 정의하는 것은 궁극적으로 메서드의 형태이다.

#어떤 새가 오리처럼 걷고 오리처럼 꽥꽥 소리를 낸다면 오리여야만 한다.


 

파이썬 3(PEP-3119) 에서 인터페이스를 덕 타이핑이 아닌 다른 방식으로 정의하는 추상 기본 클래스 개념을 도입했다.

추상 기본 클래스는 파생 클래스가 구현해야 할 일부분을 기본 동작 또는 인터페이스로 정의하는 것이다. 이는 특정 중요 메서드가 실제로 재정의 되었는지 확인이 필요할 때 유용하며 isinstance() 와 같은 메서드의 기능을 재정의 하거나 확장하는 메커니즘으로도 작동한다.

또한 이 모듈에는 가상 하위 클래스(virtual subclass) 라는 타입을 계층구조에 등록하는 기법이 포함되어 있다. 이것은 오리의 새로운 기준을 추가함으로써 덕 타이핑의 개념을 조금 더 확장하는 것이다.



#### 4-1. 너무 많은 일을 하는 인터페이스

In [30]:
class EventParser:
    def from_xml(self):
        pass
    def from_json(self):
        pass

어떤 클래스는 XML 메서드를 필요로 하지 않고 JSON 으로만 구성할 수 있음에도 여전히 인터페이스에서는 필요하지 않은 from_xml() 메서드를 제공한다. 이는 결합력을 매우 높여 유연성을 떨어뜨리며 클라이언트가 필요하지도 않은 메서드를 구현하도록 한다.

#### 4-2. 인터페이스는 작을수록 좋다
: 앞의 인터페이스는 각각 하나의 메서드를 가진 두 개의 다른 인터페이스로 분리하는 것이 좋다

In [31]:
class XMLEventParser:
    def from_xml(self):
        pass
class JSONEventParser:
    def from_json(self):
        pass

각 인터페이스 간의 독립성을 유지할 수 있고, 새로운 작은 객체를 사용해 모든 기능을 유연하게 조합할 수 있게 되었다.

 

SRP 와 유사하지만 ISP 는 인터페이스에 대해 이야기 한다. 이것은 행동의 추상화라고 볼 수 있다. 이 원칙을 준수하지 않으면 별개의 기능이 결합된 인터페이스를 만들게 되고, 상속된 클래스는 SRP 또한 준수할 수 없게 된다.

#### 4-3. 인터페이스는 얼마나 작아야 하는가
:
    추상 클래스이든 아니든 기본 클래스는 다른 클래스들이 확장할 수 있도록 인터페이스를 정의한다. 응집력의 관점에서 가능한 단 한 가지 일을 수행하는 작은 인터페이스여야 한다. 그러나 하나 이상의 메서드라 하더라도 적절하게 하나의 클래스에 속해 있을 수 있다. 예를 들어 컨텍스트 관리자는 __enter__ 와 __exit__ 두 가지 메서드를 필요로 한다. 이들이 반드시 함께 제공되어야 한다. 그렇지 않으면 유효한 컨텍스트 관리자가 아니기 때문이다.

#### 5. 의존성 역전
: 의존성역전원칙(DIP)은코드가깨지거나손상되는취약점으로부터보호해주는흥미로운디자
인원칙을제시한다. 의존성을역전시킨다는것은코드가세부사항이나구체적인구현에적응 하도록하지않고, 대신에 API 같은것에적응하도록하는것이다.

추상화를 통해 세부 사항에 의존하지 않도록 해야 하지만, 반대로 세부 사항은 추상화에 의존해야 한다.

 

A, B 두 객체가 상호 교류 한다고 가정해본다. A는 B의 인스턴스를 사용하지만 우리가 B 모듈을 직접 관리하지는 않는다. 외부 라이브러리 또는 다른 팀의 모듈 등을 사용하는 경우이다. 만약 코드가 B에 크게 의존하면 B코드가 변경되면 원래의 코드는 쉽게 깨지게 된다. 이를 방지하기 위해서 의존성을 뒤집어서 역전시켜야 한다.

 

즉 B가 A에 적응해야 한다.

이렇게 하려면 인터페이스를 개발하고 코드가 B 의 구체적인 구현에 의존하지 않도록 해야 한다. 대신에 정의한 인터페이스에 의존적이도록 해야 한다. 해당 인터페이스를 준수하는 것은 B 의 책임이다.

 

일반적으로 구체적인 구현이 추상 컴포넌트보다 더 자주 바뀐다. 이런 이유로 시스템이 변경, 수정 또는 확장될 것으로 예상되는 지점에 유연성을 확보하기 위해 추상화를 하는 것이다.

 

#### 5-1. 엄격한 의존의 예
: 이벤트의 모니터링 시스템의 마지막 부분은 식별된 이벤트를 데이터 수집기로 전달하여 분석하는 것이었다.  
단순하게 구현해보면 데이터를 목표지에 전송하는 이벤트 전송 클래스 Syslog를 만들면 된다.

In [ ]:
class EventStreamer:
    def __init__(self, parsed_data: str, client: Syslog):
        self.parsed_data = parsed_data
        self.client = client
        
    def stream(self):
        self.client.send(self.parsed_data)    
        
class Syslog:
    def send(data: str):
        pass

그러나 이는 저수준의 내용(Syslog)에 따라 고수준의 클래스(EventStreamer)가 변경되어야 하므로 별로 좋은 디자인이 아니다. 만약 Syslog 로 데이터를 보내는 방식이 변경되면 EventStreamer를 수정해야 한다.

#### 5-2. 의존성을 거꾸로
: 이러한 문제를 해결하려면 EventStreamer를 구체 클래스가 아닌 인터페이스와 대화하도록 하는 것이 좋다. 이렇게 하면 인터페이스의 구현은 세부 구현사항을 가진 저수준 클래스가 담당하게 된다.

In [ ]:
from abc import ABCMeta, abstractmethod

class EventStreamer:
    def __init__(self, parsed_data: str, client: DataTargetClient):
        self.parsed_data = parsed_data
        self.client = client
        
    def stream(self):
        self.client.send(self.parsed_data)    


class DataTargetClient(metaclass=ABCMeta):
    @abstractmethod
    def send(self, data: str):
        pass

class Syslog(DataTargetClient):
    def send(data: str):
        pass

위와 같이 데이터를 전송할 대상을 나타내는 인터페이스를 만들었다. EventStreamer는 특정 데이터 대상의 구체적인 구현과 관련이 없어졌다. 구현 내용이 바뀌어도 수정할 필요가 없다.

첫 번째 EventStreamer 구현은 Syslog 객체와만 동작했기 때문에 유연성이 떨어진다. 그렇지만 .send() 메서드를 구현하는 객체라면 어떤 것과도 통신할 수 있기 때문에 이것을 인터페이스의 메서드로 사용하였다. Syslog는 send() 메서드가 정의된 DataTargetClient 추상 기본 클래스를 확장한다.

따라서 DataTargetClient 를 확장하는 새로운 유형의 데이터 대상이 추가되어도 send() 메서드의 구현은 모두 새로운 클래스에 달려있게 된다.

심지어 런타임 중에도 send() 메서드를 구현한 객체의 프로퍼티를 수정해도 여전히 잘 동작한다. 이렇게 의존성을 동적으로 제공한다고 하여 의존성 주입(dependency injection) 이라고 한다.

파이썬은 동적인 타입의 언어이므로 인터페이스를 사용하지 않고도 EventStreamer 에 특정 데이터 대상 객체를 제공할 수 있다. 그러나 추상 기본 클래스를 사용하는 것은 좋은 습관이다.

덕 타이핑이 가능하면 모델의 가독성이 높아진다. 상속은 is a 관계이다. 따라서 추상 기본 클래스를 선언하고 확장함으로써 Syslog는 DataTargetClient 라고 말할 수 있다. 즉 코드 사용자는 코드를 읽고 이해할 수 있고 이것이 바로 덕 타이핑이다.



역주 : is a 는 객체 지향 프로그래밍에서 객체 간의 관계를 표현할 때 사용하는 용어이다. "The apple is a fruit" 처럼 is a 를 사용해 표현이 가능하다면 apple 과 fruit 은 상속 관계라는 뜻이다. apple 이 fruit 을 상속했으므로 "사과는 과일이다" 라고 말할 수 있다. 다만 그 반대는 성립하지 않는다.